In [1]:
#1 --- imports
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#2 --- make dataframe
df = pd.read_csv("cyberbullying.csv")
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [3]:
#3 --- make column with words

# Assuming your DataFrame is named df
# Replace 'your_dataframe' with your actual DataFrame name

df['words'] = df['tweet_text'].apply(lambda x: x.split())

In [4]:
#4 --- makes column with individual characters
df['characters'] = df['tweet_text'].apply(list)

In [11]:
#5 --- Most common letter

def get_top_characters(text):
    # Removing spaces from the text
    text = text.replace(" ", "")
    
    # Counting the occurrences of each character
    char_counts = {}
    for char in text:
        char_counts[char] = char_counts.get(char, 0) + 1
    
    # Sorting the characters by count in descending order
    sorted_chars = sorted(char_counts, key=char_counts.get, reverse=True)

    # Getting the 5 most common characters
    top_characters = sorted_chars[:5]

    # If there are less than 5 unique characters, padding with spaces
    top_characters += [' '] * (5 - len(top_characters))

    return top_characters

# Applying the function to each row of the 'tweet_text' column
df['top_letters'] = df['tweet_text'].apply(lambda x: get_top_letters(x))

In [ ]:
for taco in range(len(df['top_letters'])):
    print(df['top_letters'][taco][0],
          df['top_letters'][taco][1],
          df['top_letters'][taco][2],
          df['top_letters'][taco][3],
          df['top_letters'][taco][4],df['cyberbullying_type'][taco])

In [10]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [ ]:
#-----------------------------------------------------------------------------------

In [21]:
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum([1 for x, y in zip(point1, point2) if x != y]))

# Function to implement K-nearest neighbor algorithm
def k_nearest_neighbors(training_data, test_instance, k):
    distances = []

    for index, row in training_data.iterrows():
        features = np.array(row['top_letters'])
        target = row['cyberbullying_type']
        dist = euclidean_distance(test_instance, features)
        distances.append((dist, target))

    distances.sort(key=lambda x: x[0])

    neighbors = distances[:k]

    # Count the occurrences of each class in the k-nearest neighbors
    class_votes = {}
    for neighbor in neighbors:
        class_label = neighbor[1]
        class_votes[class_label] = class_votes.get(class_label, 0) + 1

    # Return the class with the most votes
    return max(class_votes, key=class_votes.get)

# Convert letters to numerical features (binary representation)
unique_characters = set(char for sublist in df['top_letters'] for char in sublist)
character_mapping = {char: i for i, char in enumerate(unique_characters)}

for char in unique_characters:
    df[char] = df['top_letters'].apply(lambda x: 1 if char in x else 0)

# Sample test instance
test_instance = [1 if char in ['a', 'b', 'c', 'd', 'e'] else 0 for char in unique_characters]

# Example usage of k_nearest_neighbors
k_value = 100
result = k_nearest_neighbors(df, test_instance, k_value)

print(f'The predicted cyberbullying type for the test instance is: {result}')

The predicted cyberbullying type for the test instance is: not_cyberbullying


In [ ]:
# import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score





# Function to clean text data
def clean_text(text):
    # Remove emojis
    text = re.sub(r"[^\w\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r"[^\w\s]", "", text)
    return text




def preprocess(df):
    print("Starting preprocessing...")


    # Clean the text data
    print("Cleaning text data...")
    df["tweet_text"] = df["tweet_text"].apply(clean_text)
    print("Text data cleaned.")


    # Create a CountVectorizer instance
    print("Creating CountVectorizer instance...")
    vectorizer = CountVectorizer(
        max_features=5000
    )  # Adjust the number of features as needed
    print("CountVectorizer instance created.")


    # Apply the vectorizer to the tweet texts to transform them into a Bag of Words model
    print("Applying CountVectorizer to tweet texts...")
    X = vectorizer.fit_transform(df["tweet_text"])
    print("CountVectorizer applied.")


    # Use 'cyberbullying_type' as the target variable
    print("Setting target variable...")
    y = df["cyberbullying_type"]
    print("Target variable set.")


    # Convert X to an array if it's not already
    print("Converting feature matrix to array...")
    X_array = X.toarray()
    print("Feature matrix converted to array.")


    # Split the data into training and testing sets (70% train, 30% test)
    print("Splitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_array, y, test_size=0.3, random_state=42
    )
    print("Data split into training and testing sets.")


    # Reset the indexes and convert the Series to NumPy arrays for y_train and y_test
    print("Resetting indexes and converting to NumPy arrays...")
    y_train = y_train.reset_index(drop=True).to_numpy()
    y_test = y_test.reset_index(drop=True).to_numpy()
    print("Indexes reset and conversion to NumPy arrays completed.")


    print("Preprocessing completed.")
    return X_train, X_test, y_train, y_test




# Call preprocess and get the train/test splits
X_train, X_test, y_train, y_test = preprocess(df)




# Function to compute Euclidean distance
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))




# KNN algorithm
class KNN:
    def __init__(self, k=3):
        print(f"Initializing KNN with k={k}")
        self.k = k


    def fit(self, X, y):
        print("Fitting model...")
        self.X_train = X
        self.y_train = y
        print("Model fitted.")


    def predict(self, X):
        print("Making predictions...")
        y_pred = [self._predict(x) for x in X]
        print("Predictions made.")
        return np.array(y_pred)


    def _predict(self, x):
        # Compute distances
        print("Computing distances...")
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
        print("Distances computed.")


        # Sort and get k nearest neighbors
        print("Finding k nearest neighbors...")
        k_indices = np.argsort(distances)[: self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        print("k nearest neighbors found.")


        # Most common label
        print("Determining the most common label...")
        most_common = Counter(k_nearest_labels).most_common(1)
        print("Most common label determined.")
        return most_common[0][0]
        # Function to find the best k




def find_best_k(X_train, X_test, y_train, y_test, min_k=1, max_k=20, step=2):
    best_k = min_k
    best_accuracy = 0


    print("Searching for the best k...")


    for k in range(min_k, max_k + 1, step):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        predictions = knn.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        print(f"k = {k}: Accuracy = {accuracy}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k




# Uncomment below lines to use the custom KNN algorithm
# knn_custom = KNN(k=3)
# knn_custom.fit(X_train, y_train)
# predictions_custom = knn_custom.predict(X_test)
# accuracy_custom = np.sum(predictions_custom == y_test) / len(y_test)
# print(f"Custom KNN Accuracy: {accuracy_custom}")


# Find the best k value
best_k = find_best_k(X_train, X_test, y_train, y_test)


# Using scikit-learn's KNeighborsClassifier with the best k
print(f"Using scikit-learn's KNeighborsClassifier with k = {best_k}...")
knn_sklearn = KNeighborsClassifier(n_neighbors=best_k)
knn_sklearn.fit(X_train, y_train)
predictions_sklearn = knn_sklearn.predict(X_test)


# Evaluate the model
accuracy_sklearn = np.mean(predictions_sklearn == y_test)
print(f"scikit-learn KNN with k = {best_k} Accuracy: {accuracy_sklearn}")


# Using scikit-learn's KNeighborsClassifier
print("Using scikit-learn's KNeighborsClassifier...")
knn_sklearn = KNeighborsClassifier(n_neighbors=3)
knn_sklearn.fit(X_train, y_train)
predictions_sklearn = knn_sklearn.predict(X_test)


# Evaluate the model
accuracy_sklearn = np.mean(predictions_sklearn == y_test)
print(f"scikit-learn KNN Accuracy: {accuracy_sklearn}")


In [ ]:
activ(18.71499999)

In [ ]:
df.head()

In [ ]:
uniquewords=df['characters'].explode().unique()
uniquewords

In [ ]:
len(uniquewords)

In [ ]:
#
for taco in range(len(df['words'])):
    for burrito in range(len(df['words'][taco])):
        x=1

In [ ]:
unique_words_df = pd.DataFrame({'word': df['words'].explode().unique()})
unique_words_df['word_number'] = range(1, len(unique_words_df) + 1)

# Merge the original DataFrame with the unique word DataFrame
df = df.merge(unique_words_df, left_on='words', right_on='word', how='left')

# Drop unnecessary columns and reset index
df = df.drop(['word', 'words'], axis=1).sort_values(by=['tweet_text']).reset_index(drop=True)

In [ ]:
fv(1,1,1)

In [ ]:
def fv(b,h,v):
    mod

In [ ]:
#-----------------------------------------------------

In [ ]:
#activation function
def activ(a):
    e=2.718281828459045235360287471352
    if a>18.715:
        1
    elif a<-18.715:
        -1
    else:
        activ_temp_1=(e**(a)-e**(-a))/(e**(a)+e**(-a))
    return activ_temp_1

In [ ]:
input("what? ")

In [ ]:
x=input("Gimme a number: ")

for taco in range(100):
    x=x*x-x
    print(x)